# Face detection and BMI/Age/Sex prediction

## Face detection

face detection is done by `MTCNN`, which is able to detect multiple faces within an image and draw the bounding box for each faces. 

It serves two purposes for this project:
 * 1) preprocess and align the facial features of image

      ![](./img/mtcnn_face_alignment.jpg)
      
 * 2) enable prediction for multiple persons in the same image
 
     ![](./img/detect_predict_multi_faces.png)


## Multi-task prediction


pretrained model `VGG-Face` is used 


## Reference
 * MTCNN: [https://github.com/ipazc/mtcnn](https://github.com/ipazc/mtcnn)
 * VGGFace: [https://github.com/rcmalli/keras-vggface](https://github.com/rcmalli/keras-vggface)

In [1]:
mode = 'train'
model_type = 'resnet50'
model_dir = './saved_model/model_'+model_type + '.h5'
bs = 8
epochs = 20

In [2]:
import pandas as pd
import os
import json
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

from models import FacePrediction

allimages = os.listdir('./face_aligned/')
train = pd.read_csv('./train.csv')
valid = pd.read_csv('./valid.csv')

train = train.loc[train['index'].isin(allimages)]
valid = valid.loc[valid['index'].isin(allimages)]

Using TensorFlow backend.


## Training

In [3]:
es = EarlyStopping(patience=3)
ckp = ModelCheckpoint(model_dir, save_best_only=True, save_weights_only=True, verbose=1)
tb = TensorBoard('./tb/%s'%(model_type))
callbacks = [es, ckp, tb]

model = FacePrediction(img_dir = './face_aligned/', model_type = model_type)
model.define_model()
model.model.summary()
if mode == 'train':
    model.train(train, valid, bs = bs, epochs = epochs, callbacks = callbacks)
else:
    model.load_weights(model_dir)

W0116 08:41:58.738450  4168 deprecation_wrapper.py:119] From C:\Users\n174724\.conda\envs\face2bmi\lib\site-packages\mtcnn\mtcnn.py:187: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0116 08:41:58.743451  4168 deprecation_wrapper.py:119] From C:\Users\n174724\.conda\envs\face2bmi\lib\site-packages\mtcnn\mtcnn.py:193: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0116 08:41:59.400462  4168 deprecation_wrapper.py:119] From C:\Users\n174724\.conda\envs\face2bmi\lib\site-packages\mtcnn\network.py:43: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0116 08:41:59.400462  4168 deprecation_wrapper.py:119] From C:\Users\n174724\.conda\envs\face2bmi\lib\site-packages\mtcnn\layer_factory.py:88: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0116 08:41:59.405312  4168 deprecation_wrapper.py:119] From C:\Users\n174724\.conda\envs\face2bmi\lib\sit

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1/7x7_s2 (Conv2D)           (None, 112, 112, 64) 9408        input_1[0][0]                    
__________________________________________________________________________________________________
conv1/7x7_s2/bn (BatchNormaliza (None, 112, 112, 64) 256         conv1/7x7_s2[0][0]               
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 112, 112, 64) 0           conv1/7x7_s2/bn[0][0]            
__________________________________________________________________________________________________
max_poolin

W0116 08:42:15.434450  4168 deprecation_wrapper.py:119] From C:\Users\n174724\.conda\envs\face2bmi\lib\site-packages\keras\callbacks.py:850: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W0116 08:42:15.436451  4168 deprecation_wrapper.py:119] From C:\Users\n174724\.conda\envs\face2bmi\lib\site-packages\keras\callbacks.py:853: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



Epoch 1/20
153/153 [==============================] - 20s 133ms/step - loss: 20.1562 - bmi_loss: 21.5665 - age_loss: 28.8775 - sex_loss: 0.1521 - sex_acc: 0.9526 - val_loss: 12.6108 - val_bmi_loss: 13.2959 - val_age_loss: 19.6771 - val_sex_loss: 0.0633 - val_sex_acc: 0.9803

Epoch 00001: val_loss improved from inf to 12.61077, saving model to ./saved_model/model_resnet50.h5
Epoch 2/20
153/153 [==============================] - 16s 105ms/step - loss: 8.1647 - bmi_loss: 8.4424 - age_loss: 14.0061 - sex_loss: 0.1017 - sex_acc: 0.9747 - val_loss: 4.8397 - val_bmi_loss: 5.1625 - val_age_loss: 7.0055 - val_sex_loss: 0.0915 - val_sex_acc: 0.9638

Epoch 00002: val_loss improved from 12.61077 to 4.83970, saving model to ./saved_model/model_resnet50.h5
Epoch 3/20
153/153 [==============================] - 16s 105ms/step - loss: 3.4951 - bmi_loss: 3.6375 - age_loss: 5.8007 - sex_loss: 0.0502 - sex_acc: 0.9828 - val_loss: 4.0817 - val_bmi_loss: 4.3717 - val_age_loss: 5.8216 - val_sex_loss: 0.0226 

## Predictions

In [ ]:
model.predict('./test_aligned/trump.jpg')

In [ ]:
preds = model.predict('./test_aligned/', show_img = True)

In [ ]:
model.predict_df('./test_aligned')

In [ ]:
metrics = model.evaulate(valid)
metrics['model'] = model_type
with open('./metrics/{:s}.json'.format(model_type), 'w') as f:
    json.dump(metrics, f)
metrics

In [ ]:
preds = model.predict_faces('./test/trump.jpg')

### Westerners

In [ ]:
preds = model.predict_faces('./test_mf/the-big-bang-theory-op-netflix.jpg', color = 'blue', fontsize=10, confidence=0.95)

In [ ]:
preds = model.predict_faces('./test_mf/friends.jpg', color = 'white')

### Asians

In [ ]:
preds = model.predict_faces('./test_mf/wulinwaizhuan.jpg', color = 'white')